<h1>Loan Status Prediction</h1>

Credit Risk Models are one of the most successful applications of data science. 
Given a set of information for a user, predict how 'reliable' she is in terms of being financially stable.
These models have raised many discussions concerning privacy, racial profiling etc.

We will be using an anonymized dataset that consists of a list of a bank's users.
You may find the dataset [here](https://www.kaggle.com/zaurbegiev/my-dataset).

Let us first install sklearn and numpy libraries:

In [1]:
!pip install sklearn
!pip install numpy

Now let's import all the required libraries.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer as Imputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import SVC

from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
# Load the data to a pandas dataframe
df = pd.read_csv("credit_train.csv")
df.head()

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,14dd8831-6af5-400b-83ec-68e61888a048,981165ec-3274-42f5-a3b4-d104041a9ca9,Fully Paid,445412.0,Short Term,709.0,1167493.0,8 years,Home Mortgage,Home Improvements,5214.74,17.2,NaN,6.0,1.0,228190.0,416746.0,1.0,0.0
1,4771cc26-131a-45db-b5aa-537ea4ba5342,2de017a3-2e01-49cb-a581-08169e83be29,Fully Paid,262328.0,Short Term,NaN,NaN,10+ years,Home Mortgage,Debt Consolidation,33295.98,21.1,8.0,35.0,0.0,229976.0,850784.0,0.0,0.0
2,4eed4e6a-aa2f-4c91-8651-ce984ee8fb26,5efb2b2b-bf11-4dfd-a572-3761a2694725,Fully Paid,99999999.0,Short Term,741.0,2231892.0,8 years,Own Home,Debt Consolidation,29200.53,14.9,29.0,18.0,1.0,297996.0,750090.0,0.0,0.0
3,77598f7b-32e7-4e3b-a6e5-06ba0d98fe8a,e777faab-98ae-45af-9a86-7ce5b33b1011,Fully Paid,347666.0,Long Term,721.0,806949.0,3 years,Own Home,Debt Consolidation,8741.90,12.0,NaN,9.0,0.0,256329.0,386958.0,0.0,0.0
4,d4062e70-befa-4995-8643-a0de73938182,81536ad9-5ccf-4eb8-befb-47a4d608658e,Fully Paid,176220.0,Short Term,NaN,NaN,5 years,Rent,Debt Consolidation,20639.70,6.1,NaN,15.0,0.0,253460.0,427174.0,0.0,0.0


In [4]:
def siivoaData(data, slice=1.0):

    # Mikä on muuttuja josta olemme kiinnostuneita?
    # Haluamme tietysti ymmärtää muuttujaa lainan tilanne "Loan staus"
    # Dikotominne muuttuja kuvaa sitä maksetaanko laina takaisin vai ei.

    #Haluammeko poistaa jotain muuttujia?
    poistettavatMuuttujat = ['Loan ID','Customer ID']
    data = data.drop(poistettavatMuuttujat, axis=1)

    #Annetaan keskiarvot puutuville tietopisteille
    sarakkeet =['Current Loan Amount','Credit Score','Annual Income','Years of Credit History',
            'Months since last delinquent','Number of Open Accounts','Number of Credit Problems',
           'Current Credit Balance','Maximum Open Credit','Bankruptcies','Tax Liens']
    muuttujanTäyttäjä = Imputer()
    data[sarakkeet] = muuttujanTäyttäjä.fit_transform(data[sarakkeet])
    data[sarakkeet] = data[sarakkeet].astype(int)

    #Poistetaan vielä NaN rivit
    data=data.dropna()

#     Note:  Select only some of the data for testing purposes!
    if slice > 0 and slice < 1:
        mid_point = int(len(data)*slice)
        data = data.loc[:mid_point]


    #Valitaan muuttuja josta olemme kiinnostuneita ja koodataan se
    y = data['Loan Status']
    new_y = []
    for i in y:
        if i == 'Fully Paid':
            new_y.append(1)
        else:
            new_y.append(0)
    data = data.drop('Loan Status', axis=1)

    # Koodataan kategoriset muuttujat
    data = pd.get_dummies(data)
    #print(data.head())

    # Normalisoidaan data
    # Palautamme myös dataMean ja dataDev arvot jos haluamme syöttää koneelle uusia havaintoja
    dataMean = np.mean(data, axis=0)
    dataDev = np.std(data, axis=0)
    norm_x = (data - dataMean) / dataDev

    x = data.values #muutetaan numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    normMinMax = pd.DataFrame(x_scaled)

    return norm_x, normMinMax, data, new_y, dataMean, dataDev

xNorm, xMinMax, xNoNorm, y, xMean, xDev = siivoaData(df, slice=0.25)

In [7]:
#Check how xNorm, xMinMax and xNoNorm dataframes look like
xNorm


,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,...,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding
0,-0.355588,-0.277100,-0.269097,-1.104831,-0.082243,-0.031961,-1.036595,1.786520,-0.170618,-0.055515,...,-0.100882,-0.182492,-0.07922,-0.056902,-0.034336,-0.247254,-0.011439,-0.053729,-0.036794,-0.026425
1,-0.361355,0.000916,-0.019983,1.193942,0.510154,-1.763498,4.755970,-0.346309,-0.166301,0.013981,...,-0.100882,-0.182492,-0.07922,-0.056902,-0.034336,-0.247254,-0.011439,-0.053729,-0.036794,-0.026425
2,2.780520,-0.252859,0.988863,0.858682,-0.526541,-0.364949,1.360328,1.786520,-0.001886,-0.002142,...,-0.100882,-0.182492,-0.07922,-0.056902,-0.034336,-0.247254,-0.011439,-0.053729,-0.036794,-0.026425
3,-0.358667,-0.268010,-0.695206,-0.816093,-0.822739,-0.031961,-0.437364,-0.346309,-0.102602,-0.060285,...,-0.100882,-0.182492,-0.07922,-0.056902,-0.034336,-0.247254,-0.011439,-0.053729,-0.036794,-0.026425
4,-0.364068,0.000916,-0.019983,0.157879,-1.711335,-0.031961,0.761097,-0.346309,-0.109537,-0.053846,...,-0.100882,-0.182492,-0.07922,-0.056902,-0.034336,-0.247254,-0.011439,-0.053729,-0.036794,-0.026425
5,-0.363111,4.708258,-0.588949,-0.191830,-0.082243,-0.031961,-1.036595,-0.346309,-0.201756,-0.078620,...,-0.100882,-0.182492,-0.07922,-0.056902,-0.034336,-0.247254,-0.011439,-0.053729,-0.036794,-0.026425
6,-0.362763,-0.261192,-0.249359,-0.643104,0.213956,-1.630302,0.361610,1.786520,-0.426885,-0.078683,...,-0.100882,-0.182492,-0.07922,-0.056902,-0.034336,-0.247254,-0.011439,-0.053729,-0.036794,-0.026425
7,-0.349184,0.000916,-0.019983,-0.319665,-1.415136,-1.763498,0.761097,-0.346309,-0.254938,0.016129,...,-0.100882,-0.182492,-0.07922,-0.056902,-0.034336,-0.247254,-0.011439,-0.053729,-0.036794,-0.026425
8,-0.352333,-0.300584,1.375586,-0.004159,0.658253,-0.098559,-1.436082,-0.346309,0.334521,-0.033373,...,-0.100882,-0.182492,-0.07922,-0.056902,-0.034336,-0.247254,-0.011439,-0.053729,-0.036794,-0.026425
9,-0.362816,-0.254374,0.070380,1.683618,-0.674640,-0.031961,1.759815,-0.346309,0.896241,0.041308,...,-0.100882,-0.182492,-0.07922,-0.056902,-0.034336,-0.247254,-0.011439,-0.053729,-0.036794,-0.026425


Next we will see how we get different predictive performances by using the raw features vs the two other feature normalization techniques.

In [8]:
cases = []
# Define three cases we will study
for x in [xNorm, xMinMax, xNoNorm]:
    case = {}
    case['x_train'], case['x_test'], case['y_train'], case['y_test'] = train_test_split(x, y, test_size= 0.25, random_state=13)
    cases.append(case)

In [9]:
# Declare the classifiers we will be using
classifiers = [SGDClassifier(), LogisticRegression(), SVC()]

for i, case in enumerate(cases):
    print("Evaluating the models with data from the case #{}".format(i+1))
    for j, clf in enumerate(classifiers):
        #train model with train data
        clf.fit(X=case['x_train'], y=case['y_train'])
        #predict test data
        predictions = clf.predict(X=case['x_test'])
        #calculate the accuracy
        accuracy = accuracy_score(case['y_test'], predictions)

        print("\t Classifier #{} achieved {} accuracy on test data.".format(j+1, accuracy))

Evaluating the models with data from the case #1
	 Classifier #1 achieved 0.815803244374673 accuracy on test data.
	 Classifier #2 achieved 0.8253968253968254 accuracy on test data.
	 Classifier #3 achieved 0.8236525379382522 accuracy on test data.
Evaluating the models with data from the case #2
	 Classifier #1 achieved 0.8250479679051108 accuracy on test data.
	 Classifier #2 achieved 0.8255712541426827 accuracy on test data.
	 Classifier #3 achieved 0.8248735391592534 accuracy on test data.
Evaluating the models with data from the case #3
	 Classifier #1 achieved 0.6685853828710971 accuracy on test data.
	 Classifier #2 achieved 0.8203383917669632 accuracy on test data.
	 Classifier #3 achieved 0.778998778998779 accuracy on test data.


In [10]:
# Implement Cross Validation using Logistic Regression classifier
# Using xMinMax dataframe

clf = LogisticRegression()
# Use all of the data (slice=1.0):
xNorm, xMinMax, xNoNorm, y, xMean, xDev = siivoaData(df, slice=1)
# 5 fold cross validation
scores = cross_val_score(clf, xMinMax, y, verbose=1, cv=5)
print(scores)
print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[0.82355398 0.81817707 0.82162247 0.82286609 0.82328374]
Accuracy: 0.822 (+/- 0.004)


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.8s finished
